In [1]:
import pandas as pd

df = pd.read_csv("fightData.txt")

In [2]:
# Key: Fighter      Value: The number of matches fighter has been in

fighters = {}

for f in df["Fighter1"]: # Count fights for fighter 1

    fighters.update({f: fighters[f]+1 if f in list(fighters.keys()) else 1})

for f in df["Fighter2"]: # Count fights for fighter 2

    fighters.update({f: fighters[f]+1 if f in list(fighters.keys()) else 1})

In [3]:
# Fighters with more than number_of_fights matches are extracted

relevant_fighters = {}
number_of_fights = 19
upper_winrate = 0.85
lower_winrate = 0.15

fighters1 = df["Fighter1"]
fighters2 = df["Fighter2"]
winners = df["Winner"]

for f in fighters.keys():
    if fighters[f] > number_of_fights:
        relevant_fighters[f] = fighters[f]

In [4]:
# Finds number of wins for each fighter

wins = {}

for f in relevant_fighters.keys():
    wins[f] = 0

for i in range(len(fighters1)):

    if fighters1[i] in relevant_fighters.keys() and winners[i] == fighters1[i]:

        wins[fighters1[i]] += 1
    
    elif fighters2[i] in relevant_fighters.keys() and winners[i] == fighters2[i]:

        wins[fighters2[i]] += 1

In [5]:
# Matches between two fighters who both had at least number_of_fights matches before the match in question are saved in a file

file = open("relevantFights.txt", "w")

file.write("Row,Fighter1,Skips1,Fighter2,Skips2\n")

skips = {}

for f in relevant_fighters.keys(): # Establishing how many matches each fighter has had after the match in question to be able to find how many they have had before said match
    skips[f] = 0

for i in range(len(fighters1)): # All matches are looped over in reverse chronological order

    if fighters1[i] in relevant_fighters.keys() and fighters2[i] in relevant_fighters.keys(): # If two relevant fighters are matched up against eachother the match is saved

        fighter1_winrate = wins[fighters1[i]] / (relevant_fighters[fighters1[i]] - skips[fighters1[i]])
        fighter2_winrate = wins[fighters2[i]] / (relevant_fighters[fighters2[i]] - skips[fighters2[i]])

        # Only matches where both fighters have winrates in the range (lower_winrate, upper_winrate) are saved
        if fighter1_winrate < upper_winrate and fighter1_winrate > lower_winrate and fighter2_winrate < upper_winrate and fighter2_winrate > lower_winrate:
            file.write(str(i+2) + "," + str(fighters1[i]) + "," + str(skips[fighters1[i]]) + "," + str(fighters2[i]) + "," + str(skips[fighters2[i]]) + "\n")

        # The amount of matches played after this point increases and if the amount of previous matches falls below the threshold the fighter in question is no longer considered
        skips[fighters1[i]] += 1
        
        if relevant_fighters[fighters1[i]] - skips[fighters1[i]] < number_of_fights:
            relevant_fighters.pop(fighters1[i])

        if winners[i] == fighters1[i]:

            wins[fighters1[i]] += 1

        skips[fighters2[i]] += 1

        if relevant_fighters[fighters2[i]] - skips[fighters2[i]] < number_of_fights:
            relevant_fighters.pop(fighters2[i])

        if winners[i] == fighters2[i]:

            wins[fighters2[i]] += 1
    
    elif fighters1[i] in relevant_fighters.keys(): # If just one relevant fighter is in a match it is not saved

        skips[fighters1[i]] += 1

        if relevant_fighters[fighters1[i]] - skips[fighters1[i]] < number_of_fights:
            relevant_fighters.pop(fighters1[i])

        if winners[i] == fighters1[i]:

            wins[fighters1[i]] += 1

    elif fighters2[i] in relevant_fighters.keys():

        skips[fighters2[i]] += 1

        if relevant_fighters[fighters2[i]] - skips[fighters2[i]] < number_of_fights:
            relevant_fighters.pop(fighters2[i])

        if winners[i] == fighters2[i]:

            wins[fighters2[i]] += 1

file.close()